# **Support Vector Regression (SVR) — Módulo 4, Notebook 2/4**

---

## Índice

1. [Introdução](#introducao)
2. [Por que SVR? Intuição e Motivação](#intuicao)
3. [Cenários e Dados](#cenarios)
4. [Visualização Inicial](#visualizacao)
5. [SVR Linear e Tubo ε](#svr-linear)
6. [Kernels e Comparações](#kernels)
7. [Hiperparâmetros e Ajustes](#hiperparametros)
8. [Conclusões e Dicas](#conclusoes)

---

<a id='introducao'></a>
## **Introdução**

Depois da regressão linear múltipla, queremos algo mais flexível para capturar curvas sem perder controle sobre a complexidade. O SVR nasce do SVM de classificação, mas troca a margem de separação por um “tubo” de tolerância aos erros.

- Objetivo: encontrar uma função suave que ignore pequenos erros (dentro de ε) e só penalize desvios maiores.
- Por que agora? Porque na regressão linear vimos limitações em relações não lineares e sensibilidade a outliers. O SVR lida melhor com essas curvas sem depender apenas de polinômios de alto grau.
- O que veremos: intuição, cenários linear vs não linear, efeito de C/ε/kernel, número de vetores de suporte como indicador de complexidade, e comparação entre kernels.

Vamos construir a intuição, visualizar, testar e comparar com sklearn.

<a id='intuicao'></a>
## **Por que SVR? Intuição e Motivação**

Recapitulando rapidamente o SVM de classificação:

- **Hiperplano**: Fronteira de decisão que separa as classes no espaço de features.
- **Margem**: Distância entre o hiperplano e os pontos mais próximos de cada classe (maximizada no SVM).
- **Vetores de suporte**: Pontos que definem a margem e são críticos para posicionar o hiperplano.

![](https://www.researchgate.net/publication/359343222/figure/fig1/AS:1182277103042573@1658888235641/SVM-and-SVR-modeling-In-SVM-left-a-hyperplane-with-maximal-margin-is-constructed-to.png?fit=600%)

Para regressão, não queremos separar classes — queremos ajustar uma função $f(x)$ que aproxima $y$.

- Em vez de minimizar o erro quadrático direto, o SVR ignora erros pequenos: se $|y - f(x)| \le \epsilon$, o custo é zero. Só quando $|y - f(x)| > \epsilon$ ativamos penalização.
- Hiperparâmetros:
  - **ε (epsilon)**: largura do tubo de tolerância. Pequeno → mais SVs (risco overfitting). Grande → menos SVs (risco underfitting).
  - **C**: penalidade para pontos fora do tubo. Alto → prioriza ajuste; baixo → prioriza suavidade.
  - **Kernel**: linear ou não linear (RBF, polinomial) para mapear o espaço e capturar curvaturas.

<a id='cenarios'></a>
## **Quando usar SVR? Cenários típicos**

- **Relações não lineares moderadas**: Quando a regressão linear não captura curvaturas, mas não queremos um modelo muito complexo.
- **Robustez a ruído moderado**: A zona de tolerância $\epsilon$ ignora pequenos desvios; outliers fortes ainda contam via $C$.
- **Dados de média/alta dimensionalidade**: Kernels (RBF, polinomial) capturam padrões sem engenharia manual de features.
- **Poucos dados com controle de capacidade**: $C$ e $\epsilon$ ajudam a equilibrar viés/variância em amostras menores.
- **Necessidade de previsões suaves**: Útil quando queremos ignorar flutuações pequenas, mas reagir a erros maiores.

In [ ]:
# =======================================================
# IMPORTANDO AS BIBLIOTECAS
# =======================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

In [ ]:
# =======================================================
# DATASETS (Linear e Não Linear)
# =======================================================

# 1. Dataset Linear
X_lin, y_lin = make_regression(n_samples=400, n_features=1, noise=12, random_state=42)
y_lin = y_lin / 100  # escala mais comportada

# 2. Dataset Não Linear (seno)
X_non = np.linspace(-3, 3, 400).reshape(-1, 1)
np.random.seed(42)
y_non = np.sin(X_non[:, 0]) + np.random.normal(0, 0.15, size=X_non.shape[0])

<a id='visualizacao'></a>
## **Visualização Inicial**

Vamos primeiro observar a natureza dos nossos datasets antes de aplicar qualquer modelo.

In [ ]:
# =======================================================
# VISUALIZAÇÃO INICIAL
# =======================================================

fig, axes = plt.subplots(1, 2, figsize=(12,4))
axes[0].scatter(X_lin, y_lin, s=15, alpha=0.6)
axes[0].set_title('Dataset Linear')
axes[0].set_xlabel('X')
axes[0].set_ylabel('y')

axes[1].scatter(X_non, y_non, s=15, alpha=0.6, c='tab:orange')
axes[1].set_title('Dataset Não Linear (seno + ruído)')
axes[1].set_xlabel('X')
axes[1].set_ylabel('y')
plt.tight_layout()
plt.show()

<a id='svr-linear'></a>
#### Visualizando o Tubo ε

Vamos ajustar um SVR linear simples e desenhar o tubo ε para ver quais pontos ficam dentro e fora.

In [ ]:
# =======================================================
# SVR Linear: Tubo epsilon
# =======================================================
from sklearn.svm import SVR 

Xtr, Xte, ytr, yte = train_test_split(X_lin, y_lin, test_size=0.2, random_state=42)

svr_lin = SVR(kernel='linear', C=1.0, epsilon=0.2) 
# kernel = 'linear' para regressão linear
# C = penalidade por erro fora do tubo (quanto maior, mais rígido)
# epsilon = largura do tubo (quanto maior, mais tolerante)

svr_lin.fit(Xtr, ytr) # Treinamento do modelo

y_pred_lin = svr_lin.predict(Xte) # Predições no conjunto de teste
rmse_lin = sqrt(mean_squared_error(yte, y_pred_lin)) # RMSE
r2_lin = r2_score(yte, y_pred_lin) # R²

# Linha de predição e tubos
X_plot = np.linspace(X_lin.min(), X_lin.max(), 300).reshape(-1,1)
y_line = svr_lin.predict(X_plot)

plt.figure(figsize=(8,5))
plt.scatter(X_lin, y_lin, s=15, alpha=0.6, label='Dados')
plt.plot(X_plot, y_line, 'r', label='Função f(x)')
plt.plot(X_plot, y_line + svr_lin.epsilon, 'k--', linewidth=1, label='+ε') # Tubo epsilon
plt.plot(X_plot, y_line - svr_lin.epsilon, 'k--', linewidth=1, label='-ε')

# Destacar pontos fora do tubo
outside_mask = np.abs(y_lin - svr_lin.predict(X_lin)) > svr_lin.epsilon
plt.scatter(X_lin[outside_mask], y_lin[outside_mask], c='gold', edgecolors='k', s=40, label='Vetores de Suporte (fora)')

plt.title(f'SVR Linear (ε={svr_lin.epsilon}) - Tubo')
plt.xlabel('X')
plt.ylabel('y')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

print(f"RMSE: {rmse_lin:.3f} | R²: {r2_lin:.3f}")
print(f"Total de vetores de suporte: {len(svr_lin.support_)} ({len(svr_lin.support_)/len(Xtr)*100:.1f}% do treino)")

### Formulação Formal

### **Formulação Formal**

Agora vamos formalizar a intuição.

Suponha que temos $n$ pares $(x^{(i)}, y^{(i)})$ com $i = 1, \ldots, n$ e queremos encontrar uma função $f(x) = \mathbf{w}^T \phi(x) + b$ que seja tão simples quanto possível (baixa norma $\|\mathbf{w}\|$) e que ao mesmo tempo incorra em erro zero ou mínimo sobre os dados.

#### Objetivo: Minimizar Erro fora do Tubo + Complexidade

O SVR resolve o problema convexo (forma primal):

$$
\min_{\mathbf{w}, b, \xi, \xi^*} \quad \frac{1}{2}\|\mathbf{w}\|^2 + C\sum_{i=1}^{n} (\xi_i + \xi_i^*)
$$

sujeito a:

$$
\begin{cases}
y^{(i)} - \mathbf{w}^T \phi(x^{(i)}) - b \le \epsilon + \xi_i \\
\mathbf{w}^T \phi(x^{(i)}) + b - y^{(i)} \le \epsilon + \xi_i^* \\
\xi_i, \xi_i^* \ge 0
\end{cases}
$$

**Interpretação:**

- **$\epsilon$ (epsilon)-tubo**: Zona de tolerância onde o custo é zero (espaçamento máximo de $\pm \epsilon$ em torno de $f(x)$).
- **$\xi_i, \xi_i^*$**: Variáveis de folga que permitem violações do tubo. Se um ponto cai fora, paga $C \times$ (distância além de $\epsilon$).
- **$C$**: Penalidade. Alto $C$ >> ajuste fino aos dados (risco overfitting). Baixo $C$ >> função mais suave (risco underfitting).
- **$\frac{1}{2}\|\mathbf{w}\|^2$**: Regularização (preferência por funções suaves). Minimizar isso = maximizar a margem.

#### Forma Dual: Representação em Termos de Vetores de Suporte

Através da teoria de otimização convexa (multiplicadores de Lagrange), transformamos o problema primal em sua **forma dual**:

$$
f(x)=\sum_{i=1}^n (\alpha_i - \alpha_i^*)\, K(x^{(i)}, x) + b
$$

onde $K(x^{(i)}, x) = \phi(x^{(i)})^T \phi(x)$ é a **função kernel** (atalho para calcular produto interno em espaço mapeado sem explicitar $\phi$).

**Implicações práticas:**

1. **Dependência só do kernel**: $\mathbf{w}$ não aparece explicitamente; a solução depende de $K(\cdot, \cdot)$ e dos pesos $\alpha_i - \alpha_i^*$.
2. **Solução esparsa automática**: A maioria dos $\alpha_i - \alpha_i^* = 0$

**Significado dos coeficientes $\alpha$:**

- **$\alpha_i = 0$ e $\alpha_i^* = 0$**: Ponto dentro do tubo >> não influencia a solução
- **$\alpha_i > 0$ ou $\alpha_i^* > 0$**: Ponto é um **vetor de suporte** >> define a função

3. **Número de vetores de suporte como indicador de complexidade**: Poucos SVs = modelo simples e bem generalizado. Muitos SVs = modelo complexo ou dados ruidosos/outliers.

### Usando SKlearn

A implementação oficial já resolve a otimização dual internamente. Vamos comparar kernels e hiperparâmetros rapidamente.

In [ ]:
# =======================================================
# IMPORTAÇÕES
# =======================================================

from sklearn.svm import SVR

# =======================================================
# SEÇÃO: COMPARAÇÃO LINEAR VS RBF (DADOS LINEARES)
# =======================================================

# Dividir dados em treino e teste
Xtr_l, Xte_l, ytr_l, yte_l = train_test_split(X_lin, y_lin, test_size=0.2, random_state=0)

# Treinar SVR Linear
model_linear = SVR(kernel='linear')
model_linear.fit(Xtr_l, ytr_l)
pred_linear = model_linear.predict(Xte_l)

# Treinar SVR RBF
model_rbf = SVR(kernel='rbf')
model_rbf.fit(Xtr_l, ytr_l)
pred_rbf = model_rbf.predict(Xte_l)

# Comparar resultados
print("\n=== Desempenho SVR: Linear vs RBF (dados lineares) ===")
for name, pred, m in [('Linear', pred_linear, model_linear), ('RBF', pred_rbf, model_rbf)]:
    mse = mean_squared_error(yte_l, pred)
    rmse = sqrt(mse)
    mae = mean_absolute_error(yte_l, pred)
    r2 = r2_score(yte_l, pred)
    n_support_vectors = len(m.support_)
    print(f"\n{name}:")
    print(f"  RMSE: {rmse:.3f}")
    print(f"  MAE:  {mae:.3f}")
    print(f"  R²:   {r2:.3f}")
    print(f"  Vetores de Suporte: {n_support_vectors}")

<a id='kernels'></a>
#### Kernel Trick na Regressão

Vamos comparar ajuste de kernel linear vs RBF no dataset não linear (seno).

In [ ]:
# =======================================================
# COMPARAÇÃO: KERNELS LINEAR VS RBF (DADOS NÃO LINEARES)
# =======================================================

# Usar dados não lineares (seno)
Xtr_n, Xte_n, ytr_n, yte_n = train_test_split(X_non, y_non, test_size=0.2, random_state=42)

# Treinar SVR Linear
svr_lin_n = SVR(kernel='linear', C=1.0, epsilon=0.05)
svr_lin_n.fit(Xtr_n, ytr_n)

# Treinar SVR RBF (kernel não linear)
svr_rbf_n = SVR(kernel='rbf', C=1.0, epsilon=0.05, gamma='scale')
svr_rbf_n.fit(Xtr_n, ytr_n)

# Gerar dados para visualização
X_plot_n = np.linspace(-3, 3, 400).reshape(-1, 1)
lin_pred_curve = svr_lin_n.predict(X_plot_n)
rbf_pred_curve = svr_rbf_n.predict(X_plot_n)

<a id='hiperparametros'></a>
#### Hiperparâmetros Importantes

- C: Controle de penalização (alto >> mais ajuste, baixo >> mais suavidade).
- ε: Largura do tubo (alto >> ignora muitos erros pequenos >> pode subajustar; baixo >> tubo fino >> mais SVs >> risco overfitting).
- kernel: 'linear', 'rbf', 'poly', 'sigmoid'.
- gamma (RBF / poly / sigmoid): Influência local (alto >> captura detalhes / risco overfit; baixo >> mais suave).
- degree (poly): Grau do polinômio.

Vamos ver sensibilidade rápida em grade reduzida (dataset seno):

In [ ]:
# =======================================================
# MINI GRID SEARCH (DEMONSTRAÇÃO DE SENSIBILIDADE)
# =======================================================

print("\n=== Mini Grid Search: Sensibilidade a Hiperparâmetros ===")
print("Testando combinações de C, epsilon e gamma...\n")

param_C = [0.1, 1, 10]
param_eps = [0.01, 0.05, 0.2]
param_gamma = ['scale', 0.5, 2]

results = []
for C in param_C:
    for eps in param_eps:
        for gamma in param_gamma:
            svr_tmp = SVR(kernel='rbf', C=C, epsilon=eps, gamma=gamma)
            svr_tmp.fit(Xtr_n, ytr_n)
            pred = svr_tmp.predict(Xte_n)
            r2 = r2_score(yte_n, pred)
            rmse = sqrt(mean_squared_error(yte_n, pred))
            results.append({'C': C, 'epsilon': eps, 'gamma': gamma, 'R2': r2, 'RMSE': rmse, 'SVs': len(svr_tmp.support_)})

# Mostrar top 10 configurações
res_df = pd.DataFrame(results).sort_values('R2', ascending=False)
print("Top 10 configurações:")
print(res_df.head(10))

*O que a saída nos diz sobre a sensibilidade aos hiperparâmetros?*

#### Otimização de Hiperparâmetros com Grid Search

Para encontrar a melhor combinação de hiperparâmetros de forma sistemática, utilizaremos `GridSearchCV` junto com a validação cruzada. Isso nos permitirá testar diferentes kernels, valores de `C`, `epsilon` e `gamma` para identificar a configuração que proporciona o melhor desempenho.


In [ ]:
# =======================================================
# IMPORTAÇÕES ADICIONAIS
# =======================================================

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# =======================================================
# GRID SEARCH: OTIMIZAR HIPERPARÂMETROS
# =======================================================

# Usar dados já criados nas células 3-4 (X_non, y_non)
# Dividir em treino e teste
Xtr_n, Xte_n, ytr_n, yte_n = train_test_split(X_non, y_non, test_size=0.2, random_state=42)

# Criar pipeline: Escalar dados antes do SVR
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR())
])

# Definir espaço de busca de hiperparâmetros
param_grid = {
    'svr__kernel': ['rbf', 'linear'],
    'svr__C': [0.1, 1, 10, 100],
    'svr__epsilon': [0.01, 0.05, 0.1, 0.2],
    'svr__gamma': ['scale', 'auto', 0.1, 1]
}

# Executar Grid Search com validação cruzada 5-fold
print("\n=== Grid Search: Otimização de Hiperparâmetros ===")
print("Executando GridSearchCV (pode demorar alguns segundos)...\n")
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2', n_jobs=-1, verbose=1)
grid_search.fit(Xtr_n, ytr_n)

# Mostrar resultados
print("\n=== Resultados do Grid Search ===")
print(f"Melhores parâmetros: {grid_search.best_params_}")
print(f"Melhor score (R²) CV: {grid_search.best_score_:.3f}")

# Avaliar no conjunto de teste
best_svr = grid_search.best_estimator_
y_pred_best = best_svr.predict(Xte_n)
test_r2 = r2_score(yte_n, y_pred_best)
test_rmse = np.sqrt(mean_squared_error(yte_n, y_pred_best))

print(f"R² no teste: {test_r2:.3f}")
print(f"RMSE no teste: {test_rmse:.3f}")

#### Pipeline e Normalização

Assim como no SVM de classificação, escalar features normalmente melhora estabilidade (principalmente com RBF / polinomial).

In [ ]:
# =======================================================
# PIPELINE MANUAL: ESCALAR + SVR RBF
# =======================================================

print("\n=== Pipeline Manual: Escalar + SVR RBF ===")
pipe_svr = Pipeline([
    ('scaler', StandardScaler()),
    ('svr', SVR(kernel='rbf', C=1.0, epsilon=0.05))
])
pipe_svr.fit(Xtr_n, ytr_n)
pred_pipe = pipe_svr.predict(Xte_n)

print(f"R² no teste: {r2_score(yte_n, pred_pipe):.3f}")
print(f"RMSE no teste: {sqrt(mean_squared_error(yte_n, pred_pipe)):.3f}")

#### Diagnóstico: Resíduos

Mesmas análises que regressão linear: queremos resíduos sem padrão, centrados em 0. Aqui mostramos rapidamente para o modelo RBF.

In [ ]:
# =======================================================
# ANÁLISE DE RESÍDUOS
# =======================================================

print("\n=== Diagnóstico: Análise de Resíduos ===")
residuals_test = yte_n - svr_rbf_n.predict(Xte_n)

fig, axes = plt.subplots(1, 3, figsize=(14, 4))

# Gráfico 1: Resíduos vs Preditos
axes[0].scatter(svr_rbf_n.predict(Xte_n), residuals_test, s=25, alpha=0.6)
axes[0].axhline(0, color='r', linestyle='--')
axes[0].set_title('Resíduos vs Preditos')
axes[0].set_xlabel('Valores Preditos')
axes[0].set_ylabel('Resíduos')

# Gráfico 2: Histograma de Resíduos
axes[1].hist(residuals_test, bins=30, edgecolor='black', alpha=0.7)
axes[1].set_title('Distribuição dos Resíduos')
axes[1].set_xlabel('Resíduos')
axes[1].set_ylabel('Frequência')

# Gráfico 3: Q-Q Plot (Normalidade)
from scipy import stats
stats.probplot(residuals_test, dist='norm', plot=axes[2])
axes[2].set_title('Q-Q Plot (Teste de Normalidade)')

plt.tight_layout()
plt.show()

print(f"Média dos resíduos: {residuals_test.mean():.6f}")
print(f"Desvio padrão: {residuals_test.std():.6f}")

<a id='conclusoes'></a>
### **Conclusões e Dicas Práticas**

Agora que você dominou os conceitos fundamentais do SVR, aqui estão as orientações práticas para aplicação em seus projetos:

1. **Começar simples:** Ajuste primeiro um SVR linear como baseline para estabelecer ponto de partida
2. **Avaliar padrão:** Se o linear não captura bem o padrão, teste kernels não lineares (RBF primeiro)
3. **Normalizar dados:** Sempre normalize features antes de usar kernels não lineares
4. **Ajustar ε:** Comece pequeno (0.05) e aumente conforme necessário. Observe % de vetores de suporte
5. **Controlar C:** Se detectar overfitting (muitos SVs + ruído), reduza C para aumentar suavidade
6. **Tunar gamma (RBF):** Use 'scale' como ponto de partida; ajuste se resultado insatisfatório
7. **Grid Search:** Aplique apenas após validar configurações básicas (economiza tempo)
8. **Comparar modelos:** Sempre compare com regressão linear para justificar complexidade adicional

#### **Referências Principais**

Para aprofundar seus conhecimentos em SVR:

- **[Documentação Oficial SVR do Scikit-Learn](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)**
  — Referência técnica completa com todos os parâmetros, exemplos e boas práticas
  
- **[Kernel Trick Explicado](https://www.geeksforgeeks.org/machine-learning/kernel-trick-in-support-vector-classification/)**
  — Revisar o conceito matemático fundamental que possibilita SVR não linear
  
- **[Diagnóstico em Regressão](https://lamfo-unb.github.io/2019/04/13/Diagnostico-em-Regressao/)**
  — Técnicas avançadas para análise de resíduos e diagnóstico de modelos de regressão

---

<-- [**Anterior: Regressão Linear Múltipla**](01_regressao_linear_multipla.ipynb) | [**Próximo: Árvores de Regressão**](03_arvores_regressao.ipynb) -->